# **Notebook Lilo and Stitching**

# AML Competition

**Team members**: Ciro Ciaravolo (*1938321*), Daorui Dong (*2244070*), Lorenzo Musso (*2049518*), Giulia Pietrangeli (*2057291*)

# Data Preparation



### Environment Setup: Clone Repositories

This cell prepares the working directory by cloning the required project repositories from GitHub. It first ensures a clean state by removing any pre-existing directories.

In [1]:
!rm -rf /kaggle/working/Lilo-and-Stitching
!rm -rf /kaggle/working/challenge
!git clone https://github.com/Mamiglia/challenge.git
!git clone https://github.com/Bohbohbohh/Lilo-and-Stitching.git

Cloning into 'challenge'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 98 (delta 39), reused 72 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 21.03 MiB | 19.79 MiB/s, done.
Resolving deltas: 100% (39/39), done.
Cloning into 'Lilo-and-Stitching'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 69 (delta 6), reused 7 (delta 2), pack-reused 51 (from 1)
Receiving objects: 100% (69/69), 149.29 MiB | 37.57 MiB/s, done.
Resolving deltas: 100% (27/27), done.


### Data Preparation: Load and Relocate Datasets

This cell performs the initial data setup. It imports necessary libraries, defines file paths, and then copies the competition's `train.npz` and `test.clean.npz` files from the input directory to the cloned repository structure.

In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

source_train_path = '/kaggle/input/aml-competition/train/train/train.npz'
source_test_path = '/kaggle/input/aml-competition/test/test/test.clean.npz'

dest_dir = '/kaggle/working/Lilo-and-Stitching/data'
dest_train_path = os.path.join(dest_dir, 'train.npz')
dest_test_path = os.path.join(dest_dir, 'test.clean.npz')

print(f"Creating destination directory: {dest_dir}")
!mkdir -p {dest_dir}

print(f"Copy of train.npz in {dest_train_path}...")
!cp {source_train_path} {dest_train_path}

print(f"Copy of test.clean.npz in {dest_test_path}...")
!cp {source_test_path} {dest_test_path}

print("Copy completed.")

Creating destination directory: /kaggle/working/Lilo-and-Stitching/data
Copy of train.npz in /kaggle/working/Lilo-and-Stitching/data/train.npz...
Copy of test.clean.npz in /kaggle/working/Lilo-and-Stitching/data/test.clean.npz...
Copy completed.


### Configuration: Update System Path

This cell appends the cloned repository's root directory to the Python system path (`sys.path`). This action ensures that custom modules and scripts within the `Lilo-and-Stitching` project can be successfully imported by the notebook.

In [3]:
import sys

repo_path = '/kaggle/working/Lilo-and-Stitching'
if repo_path not in sys.path:
    sys.path.append(repo_path)

# Models

### RMLPA Model Execution: Configuration and Workflow

This cell orchestrates the entire RMLPA (Residual Bottleneck Adapter) model workflow. It imports necessary libraries and custom utilities, then defines all key constants, hyperparameters (e.g., `BATCH_SIZE`, `LEARNING_RATE`), and file paths.

The main execution block:
1.  Sets up the environment (device and random seed).
2.  Loads, preprocesses, and splits the data into training, validation, and test sets.
3.  Creates PyTorch `DataLoaders`.
4.  Initializes the adapter model, optimizer, and scheduler.
5.  Conditionally runs the training loop based on the `PERFORM_TRAINING` flag. If `False`, it skips training and loads an existing checkpoint.
6.  Runs verification on the test/validation data.
7.  Generates the intermediate submission files using the trained adapter.

In [4]:
import torch
import numpy as np
from pathlib import Path
import os 

from utils_rmlpa import (
    setup_environment,
    load_and_prepare_data,
    create_train_val_test_splits,
    create_dataloaders,
    setup_model_and_optimizer,
    train_loop,
    run_verification,
    generate_submission_files,
    ResidualBottleneckAdapter
)

BASE_DIR = Path('/kaggle/working/Lilo-and-Stitching/')  
DATA_DIR = BASE_DIR / 'data' 
SUBMISSION_DIR = BASE_DIR / "submission" 
CHECKPOINT_DIR = BASE_DIR / "checkpoints"

TRAIN_DATA_PATH = DATA_DIR / 'train.npz'
TEST_DATA_PATH = DATA_DIR / 'test.clean.npz' 
CHECKPOINT_PATH = CHECKPOINT_DIR / "rmlpa_best_model.pth" 

PERFORM_TRAINING = True 

SEED = 42
DIM_ROBERTA = 1024
DIM_DINO = 1536

BATCH_SIZE = 512       
LEARNING_RATE = 1e-4   
WEIGHT_DECAY = 1e-4    
NUM_EPOCHS = 200
PATIENCE = 20          
NUM_WORKERS = 2 
    
D_BOTTLE_RATIO = 4     
DROPOUT_P = 0.1
INIT_TEMPERATURE = 2.6592 

TEMP_SPLIT_RATIO = 0.10  
TEST_SPLIT_RATIO_OF_TEMP = 0.50 

if __name__ == '__main__':
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    os.makedirs(SUBMISSION_DIR, exist_ok=True)
    device = setup_environment(SEED)

    X_FINAL, Y_FINAL, groups, \
    Y_gallery_unique_ALL, groups_gallery_unique_ALL = load_and_prepare_data(
        TRAIN_DATA_PATH, SUBMISSION_DIR, device
    )

    train_indices, val_indices, test_indices, \
    groups_val, groups_test = create_train_val_test_splits(
        X_FINAL, groups, TEMP_SPLIT_RATIO, TEST_SPLIT_RATIO_OF_TEMP, SEED
    )

    train_loader, val_loader, test_loader = create_dataloaders(
        X_FINAL, Y_FINAL, train_indices, val_indices, test_indices,
        BATCH_SIZE, NUM_WORKERS
    )

    adapter, temperature, optimizer, scheduler = setup_model_and_optimizer(
        DIM_ROBERTA, DIM_DINO, D_BOTTLE_RATIO, DROPOUT_P,
        INIT_TEMPERATURE, LEARNING_RATE, WEIGHT_DECAY,
        NUM_EPOCHS, len(train_loader), device
    )

    if PERFORM_TRAINING:
        train_loop(
            NUM_EPOCHS, adapter, train_loader, val_loader,
            optimizer, scheduler, temperature, device,
            groups_val, Y_gallery_unique_ALL, groups_gallery_unique_ALL,
            PATIENCE, CHECKPOINT_PATH
        )
    else:
        print(f"\nTraining skipped. Loading existing model for verification and submission.")
    
    run_verification(
        adapter, CHECKPOINT_PATH, test_loader, val_loader, device,
        groups_test, groups_val, Y_gallery_unique_ALL, groups_gallery_unique_ALL,
        SUBMISSION_DIR
    )

    generate_submission_files(
        adapter, CHECKPOINT_PATH, TEST_DATA_PATH,
        SUBMISSION_DIR, device, BATCH_SIZE
    )

    print("\nRMLPA process complete.")

--- Using device: cpu ---
Loading Training Data...
(125000,)
Train data: 125000 captions, 125000 images
Training data loaded.
Creating Unique Global Gallery...
Global Gallery created.
Gallery saved for tuning.

Preparing data...
Split 1: Creating Training set (90%) and Temporary set (10%) set...
Split 2: Splitting Temporary set into Validation set (5%) and Test set (5%)...
Split completed:
    Training set: 112500
    Validation set: 6250
    Test set: 6250
Creating Datasets and DataLoaders...
DataLoaders created and ready.
Initializing model, loss, and optimizer...
Setup complete. 
Ready for training.
    Starting Training (InfoNCE Loss)


Epoch 1/200 [Train]: 100%|██████████| 219/219 [00:17<00:00, 12.20it/s, loss=3.49]


Epoch 1 | Mode: DML (InfoNCE) | Avg Loss: 4.1551 | Val MRR: 0.7143 | LR: 1.0e-04
  -> New best MRR! Saving model to /kaggle/working/Lilo-and-Stitching/checkpoints/rmlpa_best_model.pth


Epoch 2/200 [Train]: 100%|██████████| 219/219 [00:17<00:00, 12.26it/s, loss=3.28]


Epoch 2 | Mode: DML (InfoNCE) | Avg Loss: 3.3505 | Val MRR: 0.7578 | LR: 1.0e-04
  -> New best MRR! Saving model to /kaggle/working/Lilo-and-Stitching/checkpoints/rmlpa_best_model.pth


...

Epoch 80 | Mode: DML (InfoNCE) | Avg Loss: 0.3297 | Val MRR: 0.8546 | LR: 6.6e-05
  -> MRR did not improve (20/20)
Early Stopping: MRR has not improved for 20 epochs.
Training completed. Best MRR: 0.8638

Starting Verification on Internal Test Set...
Loading best model from /kaggle/working/Lilo-and-Stitching/checkpoints/rmlpa_best_model.pth...


MRR on Internal Test Set: 0.8625

Generating validation embeddings for alpha tuning...


Validation embeddings generated.

Starting Submission File Generation...
Loading model...
Model loaded.
Loading test data...
Test data loaded.
Running inference on submission data...


[Submission] Execution R-MLP-A: 100%|██████████| 3/3 [00:00<00:00, 27.68it/s]


Submission embeddings for ensemble saved.
Calculating similarity and saving submission...
Generating submission file...
✓ Saved submission to /kaggle/working/Lilo-and-Stitching/submission/submission_rmlpa.csv
Submission created successfully!

RMLPA process complete.


### MLP Model Execution: Configuration and Workflow

This cell orchestrates the entire MLP (Multi-Layer Perceptron) model workflow, which is configured for triplet loss. It imports necessary libraries and custom utilities from `utils_mlp`, then defines all key constants, hyperparameters (e.g., `BATCH_SIZE`, `N_EPOCHS_FINALI`), and file paths.

The main execution block:
1.  Sets up the environment (device, random seed, and directories).
2.  Loads and preprocesses the data, splitting it into training, validation, and internal test sets.
3.  Initializes a custom `TripletDataset` and `DataLoader` for training.
4.  Initializes the `LatentMapper` model, triplet loss function, optimizer, and scheduler.
5.  Conditionally runs the training loop based on the `PERFORM_TRAINING` flag. If `False`, it skips training.
6.  Loads the best model checkpoint from disk.
7.  Runs validation, saves embeddings, and performs an internal test.
8.  Loads the official competition test data and generates the intermediate submission files.

In [5]:
import torch
import numpy as np
from pathlib import Path
import os
from torch.utils.data import DataLoader

from utils_mlp import (
    setup_paths_and_device,
    load_and_prepare_data_mlp,
    load_submission_test_data,
    setup_model_and_optimizer_mlp,
    TripletDataset,
    run_training_loop,
    save_validation_embeddings,
    run_internal_test,
    generate_dml_submission,
    LatentMapper,
    set_seed
)

PERFORM_TRAINING = True 

SEED = 42
BATCH_SIZE = 512
N_EPOCHS_FINALI = 200
PATIENCE = 20
NUM_WORKERS = 0 

VAL_SPLIT_SIZE = 0.05 
TEST_SPLIT_SIZE = 0.05
TEMP_SPLIT_RATIO = VAL_SPLIT_SIZE + TEST_SPLIT_SIZE 
TEST_SPLIT_RATIO_OF_TEMP = TEST_SPLIT_SIZE / TEMP_SPLIT_RATIO 

BASE_DIR_PATH = Path('/kaggle/working/Lilo-and-Stitching/') 
DATA_DIR_NAME = 'data'
SUBMISSION_DIR_NAME = 'submission'
CHECKPOINT_DIR_NAME = 'checkpoints'
MODEL_FILE_NAME = 'final_latent_mapper.pth'

if __name__ == '__main__':

    set_seed(42)

    DEVICE, BASE_DIR, DATA_DIR, \
    SUBMISSION_DIR, CHECKPOINT_DIR, \
    FINAL_MODEL_PATH = setup_paths_and_device(
        BASE_DIR_PATH, CHECKPOINT_DIR_NAME, 
        SUBMISSION_DIR_NAME, MODEL_FILE_NAME
    )
    
    TRAIN_DATA_PATH = DATA_DIR / 'train.npz'
    TEST_DATA_PATH = DATA_DIR / 'test.clean.npz'

    data_pack = load_and_prepare_data_mlp(
        TRAIN_DATA_PATH, TEMP_SPLIT_RATIO, 
        TEST_SPLIT_RATIO_OF_TEMP, SEED, DEVICE
    )
    
    X_train_P_tensor, Y_train_P_tensor = data_pack["train"]
    X_val_tensor, Y_val_tensor = data_pack["val"]
    X_test_P_tensor, Y_test_split_np = data_pack["test"]
    groups_val = data_pack["groups_val"]
    groups_test = data_pack["groups_test"]
    init_stats = data_pack["init_stats"]
    PADDING_SIZE = data_pack["PADDING_SIZE"]
    D_vae = data_pack["D_vae"]

    train_dataset = TripletDataset(X_train_P_tensor, Y_train_P_tensor)
    g = torch.Generator()
    g.manual_seed(SEED)
    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, 
        shuffle=True, num_workers=NUM_WORKERS,
        generator=g
    )
    
    model, triplet_loss, optimizer, scheduler = setup_model_and_optimizer_mlp(
        D_vae, init_stats, N_EPOCHS_FINALI, len(train_loader), DEVICE
    )

    if PERFORM_TRAINING:
        run_training_loop(
            model=model,
            train_loader=train_loader,
            optimizer=optimizer,
            scheduler=scheduler,
            triplet_loss=triplet_loss,
            N_EPOCHS=N_EPOCHS_FINALI, 
            FINAL_MODEL_PATH=FINAL_MODEL_PATH,
            X_val=X_val_tensor,         
            Y_val=Y_val_tensor,  
            groups_val=groups_val,       
            PATIENCE=PATIENCE,          
            DEVICE=DEVICE
        )
    else:
        print(f"\nTraining skipped. Loading model from {FINAL_MODEL_PATH} ---")

    if not os.path.exists(FINAL_MODEL_PATH):
        print(f"ERROR: Checkpoint not found at{FINAL_MODEL_PATH}")
    else:
        model.load_state_dict(torch.load(FINAL_MODEL_PATH, map_location=DEVICE))
        model.eval()

        save_validation_embeddings(
            model, X_val_tensor, groups_val, SUBMISSION_DIR
        )

        run_internal_test(
            model, X_test_P_tensor, Y_test_split_np, groups_test, DEVICE
        )

        X_test_np, test_data_ids = load_submission_test_data(TEST_DATA_PATH)

        generate_dml_submission(
            model=model,
            FINAL_MODEL_PATH=FINAL_MODEL_PATH,
            X_test_np=X_test_np,
            test_data_ids=test_data_ids,
            PADDING_SIZE=PADDING_SIZE,
            BASE_DIR=BASE_DIR,
            submission_suffix="mlp",
            DEVICE=DEVICE
        )

        print("\nMLP process complete.")

--- Using device: cpu ---
Loading Training data...
(125000,)
Train data: 125000 captions, 125000 images
Training data loaded.

Preparing data...
Split 1: Creating Training set (90%) and Temporary set (10%) set...
Split 2: Splitting Temporary set into Validation set (5%) and Test set (5%)...
Split completed:
    Training set: 112500
    Validation set: 6250
    Test set: 6250
R, bias, mu_x calculated.

DML model initialisation...
Loss: Triplet Loss (Margin=0.2) | Optimizer: AdamW (lr=1e-05)

Start Training for 200 epochs with Early Stopping (Patience=20)...


Epoch 1/200 [Train]: 100%|██████████| 220/220 [00:24<00:00,  9.16it/s, Loss=0.199]


Epoch 1/200 | Train Loss: 0.208822 | Val MRR: 0.759346
New Best Model Saved! Val MRR: 0.759346


Epoch 2/200 [Train]: 100%|██████████| 220/220 [00:23<00:00,  9.22it/s, Loss=0.199]


Epoch 2/200 | Train Loss: 0.204222 | Val MRR: 0.783626
New Best Model Saved! Val MRR: 0.783626


...

Epoch 150/200 [Train]: 100%|██████████| 220/220 [00:24<00:00,  8.85it/s, Loss=0.175]


Epoch 150/200 | Train Loss: 0.178889 | Val MRR: 0.880769
Patience: 19/20


Epoch 151/200 [Train]: 100%|██████████| 220/220 [00:22<00:00,  9.61it/s, Loss=0.173]


Epoch 151/200 | Train Loss: 0.178813 | Val MRR: 0.880135
Patience: 20/20
Early stopping triggered at epoch 151. MRR does not improve after 20 epochs.

Training completed.
Best model saved  with MRR Val: 0.882917


MRR on Internal Test Set: 0.884036

Loading Test data...
Test data loaded.
Embedding for ensembles saved.
Generating submission file...
✓ Saved submission to /kaggle/working/Lilo-and-Stitching/submission/submission_mlp.csv
DML submission saved.

MLP process complete.


### Stitcher Model Execution: Configuration and Workflow

This cell orchestrates the entire Stitcher model workflow. It imports necessary libraries and custom utilities from `utils_stitcher`, then defines all key constants, hyperparameters (e.g., `BATCH_SIZE`, `LEARNING_RATE`, `MARGIN`), and file paths.

The main execution block:
1.  Sets up the environment (device and random seed).
2.  Loads, preprocesses, and splits the data into training, validation, and test sets.
3.  Loads the global gallery.
4.  Creates PyTorch `DataLoaders`.
5.  Initializes the `Stitcher` model architecture.
6.  Conditionally runs the training loop based on the `PERFORM_TRAINING` flag. If `False`, it skips training.
7.  Loads the best model checkpoint from disk.
8.  Runs verification on the internal validation and test splits.
9.  Generates the intermediate submission files by processing the official competition test data.

In [6]:
import torch
import numpy as np
from pathlib import Path
import os 

from utils_stitcher import (
    setup_environment_stitcher,
    load_and_prepare_data_stitcher,
    create_splits_stitcher,
    load_global_gallery,
    create_dataloaders_stitcher,
    setup_model_stitcher,
    train_model_stitcher,
    run_verification_stitcher,
    generate_submission_files_stitcher,
    Stitcher 
)

BASE_DIR = Path('/kaggle/working/Lilo-and-Stitching/')  
DATA_DIR = BASE_DIR / 'data' 
SUBMISSION_DIR = BASE_DIR / "submission" 
CHECKPOINT_DIR = BASE_DIR / "checkpoints"

CHECKPOINT_PATH = CHECKPOINT_DIR / "stitcher_best_model.pth" 
TRAIN_DATA_PATH = DATA_DIR / 'train.npz'
TEST_DATA_PATH = DATA_DIR / 'test.clean.npz' 

PERFORM_TRAINING = True 

SEED = 42
DIM_ROBERTA = 1024
DIM_DINO = 1536

BATCH_SIZE = 512       
LEARNING_RATE = 1e-4   
NUM_EPOCHS = 200 
PATIENCE = 20          
NUM_WORKERS = 2 
MARGIN = 0.2       
DROPOUT_RATE = 0.5 
HIDDEN_DIM = 2048  

TEMP_SPLIT_RATIO = 0.10  
TEST_SPLIT_RATIO_OF_TEMP = 0.50 

if __name__ == '__main__':
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    os.makedirs(SUBMISSION_DIR, exist_ok=True)
    device = setup_environment_stitcher(SEED)

    X_FINAL, Y_FINAL, groups = load_and_prepare_data_stitcher(TRAIN_DATA_PATH, device)

    train_indices, val_indices, test_indices, \
    groups_val, groups_test = create_splits_stitcher(
        X_FINAL, groups, TEMP_SPLIT_RATIO, TEST_SPLIT_RATIO_OF_TEMP, SEED
    )

    Y_gallery_unique_ALL, groups_gallery_unique_ALL = load_global_gallery(SUBMISSION_DIR)

    train_loader, val_loader_train, val_loader_inf = create_dataloaders_stitcher(
        X_FINAL, Y_FINAL, train_indices, val_indices, groups,
        BATCH_SIZE, NUM_WORKERS
    )

    model = setup_model_stitcher(
        DIM_ROBERTA, DIM_DINO, HIDDEN_DIM, DROPOUT_RATE, device
    )

    if PERFORM_TRAINING:
        train_model_stitcher(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader_train, 
            DEVICE=device,
            EPOCHS=NUM_EPOCHS,
            LR=LEARNING_RATE,
            MARGIN=MARGIN,
            MODEL_PATH=CHECKPOINT_PATH,
            PATIENCE=PATIENCE,
            groups_val=groups_val,
            Y_gallery_unique_ALL=Y_gallery_unique_ALL,
            groups_gallery_unique_ALL=groups_gallery_unique_ALL
        )
    else:
        print(f"\nTraining skipped. Loading existing model.\n")

    X_test_tensor = X_FINAL[test_indices]
    
    run_verification_stitcher(
        model=model,
        checkpoint_path=CHECKPOINT_PATH,
        val_loader_inf=val_loader_inf,
        X_test_tensor=X_test_tensor,
        groups_test=groups_test,
        groups_val=groups_val,
        Y_gallery_unique_ALL=Y_gallery_unique_ALL,
        groups_gallery_unique_ALL=groups_gallery_unique_ALL,
        submission_dir=SUBMISSION_DIR,
        device=device
    )

    generate_submission_files_stitcher(
        model=model,
        checkpoint_path=CHECKPOINT_PATH,
        test_data_path=TEST_DATA_PATH,
        submission_dir=SUBMISSION_DIR,
        device=device,
        batch_size=BATCH_SIZE
    )

    print("\nStitcher process complete.")

--- Using device: cpu ---
Loading training data...
(125000,)
Train data: 125000 captions, 125000 images
Training data loaded.

Preparing data...
Split 1: Creating Training set (90%) and Temporary set (10%) set...
Split 2: Splitting Temporary set into Validation set (5%) and Test set (5%)...
Split completed:
    Training set: 112500
    Validation set: 6250
    Test set: 6250
Global gallery loaded.
Creating Datasets and DataLoaders...
DataLoaders loaded and ready.
Initializing model...
    Stitcher parameters: 11,015,680
Starting Stitcher training for 200 epochs...
Saving checkpoint in: /kaggle/working/Lilo-and-Stitching/checkpoints/stitcher_best_model.pth


Epoch 1/200 [Train]: 100%|██████████| 219/219 [01:06<00:00,  3.29it/s, loss=0.203]


Epoch 1 | Train Loss (Triplet): 0.209426 | Val MRR: 0.771456 | LR: 1.0e-04
 -> New, best MRR! Saving the model to /kaggle/working/Lilo-and-Stitching/checkpoints/stitcher_best_model.pth


Epoch 2/200 [Train]: 100%|██████████| 219/219 [01:06<00:00,  3.29it/s, loss=0.198]


Epoch 2 | Train Loss (Triplet): 0.200344 | Val MRR: 0.824144 | LR: 1.0e-04
 -> New, best MRR! Saving the model to /kaggle/working/Lilo-and-Stitching/checkpoints/stitcher_best_model.pth


...

Epoch 37 | Train Loss (Triplet): 0.152611 | Val MRR: 0.878332 | LR: 9.2e-05
  -> MRR not improved (20/20)
--- Early Stopping: MRR hasn't improved in 20 epochs. ---

Training completed. The best model has been saved at /kaggle/working/Lilo-and-Stitching/checkpoints/stitcher_best_model.pth with Val MRR: 0.882926
Loading best model...


[Validation] Stitcher execution: 100%|██████████| 7/7 [00:01<00:00,  4.49it/s]


MRR on Internal Test Set: 0.881959
Loading Test data...
Test data loaded.


[Submission] Stitcher execution: 100%|██████████| 3/3 [00:00<00:00,  6.57it/s]


Embeddings for ensemble saved.
Generating submission file...
✓ Saved submission to /kaggle/working/Lilo-and-Stitching/submission/submission_stitcher.csv
Submission successfully created!

Stitcher process complete.


# Ensemble

### Ensemble Weight Tuning: Validation Search

This cell determines the optimal weights for combining the embeddings from the different models. It searches for the combination that maximizes the Mean Reciprocal Rank (MRR) on the validation set.

1.  **Setup and Data Loading:**
    * Imports necessary libraries (`numpy`, `sklearn`, `tqdm`, etc.) and sets up system and submission paths.
    * Imports the custom `calculate_mrr_validation_sampled` function.
    * Loads the pre-computed embeddings and groups for `val_rmlpa`, `val_mlp`, `val_stitcher`, and the `gallery_data`.

2.  **2-Model Ensemble (RMLPA + Stitcher):**
    * Performs a **linear search** over values (from 0.0 to 1.0) for the `alpha` weight.
    * For each `alpha`, it calculates the weighted average: `alpha * emb_rmlpa + (1 - alpha) * emb_stitcher`.
    * **Normalizes (L2)** the resulting combined embedding.
    * Calculates the MRR on the validation set using the `calculate_mrr_validation_sampled` function.
    * Records and prints the best `alpha` (weight for RMLPA) and the highest MRR achieved.

3.  **3-Model Weight Calculation (RMLPA + MLP + Stitcher):**
    * It calculates a set of weights based on the inverse of predefined overfitting (`Local_MRR` - `Training_MRR`) constants (`O_RMLPA_VAL`, `O_MLP_VAL`, `O_STITCHER_VAL`).
    * The calculated weights (`rob_r`, `rob_m`, `rob_s`) are normalized (so their sum is 1.0) and printed.

In [7]:
import numpy as np
from sklearn.preprocessing import normalize
from tqdm import tqdm
import itertools
from pathlib import Path
import sys

BASE_DIR = Path('/kaggle/working/Lilo-and-Stitching')
if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))
SUBMISSION_DIR = BASE_DIR / "submission"

from utils_tuning import calculate_mrr_validation_sampled

print("Calculating 2-Model AND 3-Model ensembles...")

try:
    print("Loading models...")
    data_rmlpa = np.load(SUBMISSION_DIR / "val_rmlpa.npz")
    emb_rmlpa = data_rmlpa['embeddings']
    groups_val = data_rmlpa['groups']
    print("RMLPA loaded.")

    data_mlp = np.load(SUBMISSION_DIR / "val_mlp.npz")
    emb_mlp = data_mlp['embeddings']
    print("MLP loaded.")
    
    data_stitcher = np.load(SUBMISSION_DIR / "val_stitcher.npz")
    emb_stitcher = data_stitcher['embeddings']
    print("Stitcher loaded.")

    gallery_data = np.load(SUBMISSION_DIR / "gallery_data.npz")
    Y_gallery = gallery_data['embeddings']
    groups_gallery = gallery_data['groups']
    print("Gallery loaded.\n")

except FileNotFoundError as e:
    print(f"CRITICAL ERROR: File missing -> {e}")
    print("   Cannot proceed without base validation files (RMLPA, Stitcher, Gallery).")
    sys.exit()

print("\nStarting 2-Model Tuning (RMLPA + Stitcher)...")

best_mrr_2m = -1.0
best_alpha = -1.0
alphas = np.linspace(0.0, 1.0, 21)

for alpha in tqdm(alphas, desc="Linear Search 2-Model"):
    emb_final = (alpha * emb_rmlpa) + ((1 - alpha) * emb_stitcher)
    emb_final = normalize(emb_final, axis=1, norm='l2')
    
    mrr = calculate_mrr_validation_sampled(emb_final, groups_val, Y_gallery, groups_gallery)
    
    if mrr > best_mrr_2m:
        best_mrr_2m = mrr
        best_alpha = alpha 

print("\n2-Model Results")
print(f"BEST ALPHA (RMLPA Weight): {best_alpha:.2f}")
print(f"   (Resulting Stitcher Weight: {1.0 - best_alpha:.2f})")
print(f"MRR: {best_mrr_2m:.6f}")

print("\nStarting 3-Model Tuning (RMLPA + MLP + Stitcher)...")

try:

    O_RMLPA_VAL    = 0.0007
    O_STITCHER_VAL = 0.00038
    O_MLP_VAL      = 0.004057

    W_r, W_m, W_s = 1/O_RMLPA_VAL, 1/O_MLP_VAL, 1/O_STITCHER_VAL
    W_tot = W_r + W_m + W_s
    rob_r, rob_m, rob_s = W_r/W_tot, W_m/W_tot, W_s/W_tot
    print(f"\nWEIGHTS: R={rob_r:.3f}, M={rob_m:.3f}, S={rob_s:.3f}")

except FileNotFoundError:
    print(f"\nWARNING: File 'val_mlp.npz' not found.")
    print("   Skipping 3-model tuning.")
except Exception as e:
    print(f"\nUnexpected ERROR during 3-model tuning: {e}")

print("\nTuning completed.")

Calculating 2-Model AND 3-Model ensembles...
Loading models...
RMLPA loaded.
MLP loaded.
Stitcher loaded.
Gallery loaded.


Starting 2-Model Tuning (RMLPA + Stitcher)...


Linear Search 2-Model: 100%|██████████| 21/21 [01:50<00:00,  5.25s/it]


2-Model Results
BEST ALPHA (RMLPA Weight): 0.20
   (Resulting Stitcher Weight: 0.80)
MRR: 0.882485

Starting 3-Model Tuning (RMLPA + MLP + Stitcher)...

WEIGHTS: R=0.332, M=0.057, S=0.611

Tuning completed.


### Final Ensemble Generation and Submission

This cell creates the final competition-ready submission files by ensembling the model outputs. It first loads the pre-computed test embeddings from the individual RMLPA, Stitcher, and MLP models.

Based on pre-defined weights, it then generates two distinct submission files:
1.  **2-Model Ensemble:** A weighted average of the **RMLPA** and **Stitcher** embeddings is calculated, L2-normalized, and saved to a `.csv` file.
2.  **3-Model Ensemble:** A separate weighted average of the **RMLPA**, **MLP**, and **Stitcher** embeddings is calculated, L2-normalized, and saved to a second `.csv` file.

In [8]:
import numpy as np
import os
import sys
from pathlib import Path
from sklearn.preprocessing import normalize

ALPHA = best_alpha

W_RMLPA    = rob_r
W_MLP      = rob_m
W_STITCHER = rob_s

BASE_DIR = Path('/kaggle/working/Lilo-and-Stitching')
if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))
SUBMISSION_DIR = BASE_DIR / "submission"
os.makedirs(SUBMISSION_DIR, exist_ok=True)

try:
    from challenge.src.common.utils import generate_submission
except ImportError:
    try:
        from utils_rmlpa import generate_submission
    except ImportError:
        def generate_submission(ids, embeddings, output_file):
            import pandas as pd
            df = pd.DataFrame(embeddings, index=ids)
            df.to_csv(output_file)
            print(f"WARNING: Using fallback save function for: {output_file}")

print("\nLoading models...")
try:
    data_r = np.load(SUBMISSION_DIR / "submission_rmlpa.npz")
    emb_r = data_r['embeddings']
    ids = data_r['ids'] 
    print("RMLPA loaded.")

    data_m = np.load(SUBMISSION_DIR / "submission_mlp.npz")
    emb_m = data_m['embeddings']
    print("MLP loaded.")

    data_s = np.load(SUBMISSION_DIR / "submission_stitcher.npz")
    emb_s = data_s['embeddings']
    print("Stitcher loaded.")

except FileNotFoundError as e:
    print(f"ERROR: File not found -> {e}")
    print("Ensure 'submission_rmlpa.npz', 'submission_stitcher.npz', and 'submission_mlp.npz' are all present.")
    sys.exit()
except Exception as e:
    print(f"Unexpected ERROR during loading: {e}")
    sys.exit()

print("\nStarting 2-Model Ensemble Calculation")
print(f"Configuration: RMLPA={ALPHA:.3f}, STITCHER={1-ALPHA:.3f}")

emb_final_2 = (ALPHA * emb_r) + ((1 - ALPHA) * emb_s)

emb_final_2 = normalize(emb_final_2, axis=1, norm='l2')
print("L2 normalization complete.")

csv_name_2 = f"submission_ensemble_2model_ALPHA_{ALPHA:.3f}.csv"
csv_path_2 = SUBMISSION_DIR / csv_name_2

print(f"Generating CSV - submission (2-model): {csv_path_2.name} ...")
generate_submission(ids, emb_final_2, str(csv_path_2))
print("2-Model Ensemble complete.")

print("\nStarting 3-Model Ensemble Calculation")

tot = W_RMLPA + W_MLP + W_STITCHER
w_r, w_m, w_s = W_RMLPA/tot, W_MLP/tot, W_STITCHER/tot

print(f"Configuration: RMLPA={w_r:.3f}, MLP={w_m:.3f}, STITCHER={w_s:.3f}")

emb_final_3 = (w_r * emb_r) + (w_m * emb_m) + (w_s * emb_s)

emb_final_3 = normalize(emb_final_3, axis=1, norm='l2')
print("L2 normalization complete.")

csv_name_3 = "submission_ensemble_3model.csv"
csv_path_3 = SUBMISSION_DIR / csv_name_3

print(f"Generating CSV - submission (3-model): {csv_path_3.name} ...")
generate_submission(ids, emb_final_3, str(csv_path_3))
print("3-Model Ensemble complete.")


print("\n\nOperation complete! Both ensembles created.")


Loading models...
RMLPA loaded.
MLP loaded.
Stitcher loaded.

Starting 2-Model Ensemble Calculation
Configuration: RMLPA=0.200, STITCHER=0.800
L2 normalization complete.
Generating CSV - submission (2-model): submission_ensemble_2model_ALPHA_0.200.csv ...
Generating submission file...
✓ Saved submission to /kaggle/working/Lilo-and-Stitching/submission/submission_ensemble_2model_ALPHA_0.200.csv
2-Model Ensemble complete.

Starting 3-Model Ensemble Calculation
Configuration: RMLPA=0.332, MLP=0.057, STITCHER=0.611
L2 normalization complete.
Generating CSV - submission (3-model): submission_ensemble_3model.csv ...
Generating submission file...
✓ Saved submission to /kaggle/working/Lilo-and-Stitching/submission/submission_ensemble_3model.csv
3-Model Ensemble complete.


Operation complete! Both ensembles created.
